In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_json('/content/drive/My Drive/Colab Notebooks/IMDB_reviews.json' , lines =True)

In [6]:
df.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"


In [7]:
df["is_spoiler"].value_counts()


False    422989
True     150924
Name: is_spoiler, dtype: int64

In [8]:
df.isnull().values.any()

False

In [10]:
print( df['is_spoiler'].isnull().sum() )
print( df['review_text'].isnull().sum() )
print( df['review_summary'].isnull().sum() )

0
0
0


In [11]:
df["full_summary"] = df['review_summary'] + ' ' + df['review_text']
df.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary,full_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.,A classic piece of unforgettable film-making. ...
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.,Simply amazing. The best film of the 90's. The...
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film,The best story ever told on film I believe tha...
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?,"Busy dying or busy living? **Yes, there are SP..."
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted","Great story, wondrously told and acted At the ..."


In [12]:
df['is_spoiler_label'] = np.where(df['is_spoiler'] == True, 1, 0)


In [14]:
df_use = df[["is_spoiler_label","full_summary"]].copy()
df_use

,is_spoiler_label,full_summary
0,1,A classic piece of unforgettable film-making. ...
1,1,Simply amazing. The best film of the 90's. The...
2,1,The best story ever told on film I believe tha...
3,1,"Busy dying or busy living? **Yes, there are SP..."
4,1,"Great story, wondrously told and acted At the ..."
...,...,...
573908,0,The best teen movie of the nineties Go is wise...
573909,0,"Go - see the movie Well, what shall I say. thi..."
573910,0,It's the best movie I've ever seen Go is the b...
573911,0,Haven't we seen this before? Call this 1999 te...


In [15]:
import string 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    return " ".join(without_stopwords)

In [16]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
%%time
df_use['full_summary_clean'] = df_use['full_summary'].apply(lambda x: clean(x))
df_use.head()

CPU times: user 10min 29s, sys: 8 s, total: 10min 37s
Wall time: 10min 37s


In [19]:
df_use.head()

,is_spoiler_label,full_summary,full_summary_clean
0,1,A classic piece of unforgettable film-making. ...,classic piece unforgettable film making oscar ...
1,1,Simply amazing. The best film of the 90's. The...,simply amazing best film shawshank redemption ...
2,1,The best story ever told on film I believe tha...,best story ever told film believe film best st...
3,1,"Busy dying or busy living? **Yes, there are SP...",busy dying busy living yes spoilers film emoti...
4,1,"Great story, wondrously told and acted At the ...",great story wondrously told acted heart extrao...


In [20]:
from nltk.stem.porter import PorterStemmer

def stemm (text):
  tokenized = word_tokenize(text)
  stemmer = PorterStemmer()
  stemmed = [stemmer.stem(word) for word in tokenized]
  return " ".join(stemmed)

In [21]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [22]:
df_use['full_summary_clean'] = df_use['full_summary_clean'].apply(lambda x: stemm(x))

In [23]:
df_use.head()

,is_spoiler_label,full_summary,full_summary_clean
0,1,A classic piece of unforgettable film-making. ...,classic piec unforgett film make oscar year sh...
1,1,Simply amazing. The best film of the 90's. The...,simpli amaz best film shawshank redempt withou...
2,1,The best story ever told on film I believe tha...,best stori ever told film believ film best sto...
3,1,"Busy dying or busy living? **Yes, there are SP...",busi die busi live ye spoiler film emot impact...
4,1,"Great story, wondrously told and acted At the ...",great stori wondrous told act heart extraordin...


In [25]:
df_use.drop(columns="full_summary", inplace= True)
df_use

,is_spoiler_label,full_summary_clean
0,1,classic piec unforgett film make oscar year sh...
1,1,simpli amaz best film shawshank redempt withou...
2,1,best stori ever told film believ film best sto...
3,1,busi die busi live ye spoiler film emot impact...
4,1,great stori wondrous told act heart extraordin...
...,...,...
573908,0,best teen movi nineti go wise fast pure entert...
573909,0,go see movi well shall say fun rate three plot...
573910,0,best movi ever seen go best movi ever seen see...
573911,0,seen call teenag version pulp fiction whatev w...


In [26]:
df_use.to_csv('data_cleaned_stem_v4.csv', index=False)

In [28]:
!cp data_cleaned_stem_v4.csv "/content/drive/My Drive/Colab Notebooks"

DO NOT RUN ANY CELL BELOW HERE :)


In [ ]:
### DO NOT RUUUUUUUUUUUUUUUUUUN###
# Training set sizes to loop over
train_sizes = [50,100,200,300,400,500,600]

for size in train_sizes:
    
    # Sample the training set
    data_train_sample = data_train.sample(size, random_state = 5)
    
    # Prepare X and y
    X_train = data_train_sample[['surface']]
    y_train = data_train_sample['price']
    
    # Train the model on the sampled Train data
    model = LinearRegression().fit(X_train, y_train)

    # Score the model on the train data (sampled) and test data (fixed)
    test_score =   model.score(X_test,y_test)
    train_score =  model.score(X_train,y_train)
    
    # Append associated train score, test score, and train size to dataframe
    learning_curves_elements = learning_curves_elements.append({'train_score': train_score,
                                  'test_score': test_score,
                                  'train_size': size}, ignore_index=True)
    
import matplotlib.pyplot as plt

plt.plot(learning_curves_elements.train_size,learning_curves_elements.train_score,  color = 'red', label='train score')
plt.plot(learning_curves_elements.train_size,learning_curves_elements.test_score, color = 'blue', label='test score')
plt.title('Learning Curves')
plt.xlabel('train size')
plt.ylabel('R2 score')
plt.legend()
plt.show()

In [ ]:
### DO NOT RUUUUUUUUUUUUUUUUUUN###

from sklearn.linear_model import LinearRegression

model = LinearRegression()

# 5-Fold Cross validate model
cv_results = cross_validate(model, X, y, cv=5)

# Cross validation results
cv_results

# Mean of scores
cv_results['test_score'].mean()




In [ ]:
### DO NOT RUUUUUUUUUUUUUUUUUUN###

from sklearn.model_selection import cross_val_predict

log_model = LinearRegression()

y_pred = cross_val_predict(log_model, X, y, cv=5)

In [ ]:
### DO NOT RUUUUUUUUUUUUUUUUUUN###


from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Encode the target
data['price_range_encoded'] = LabelEncoder().fit_transform(data['price_range'])

# Make predictions using a LogisticRegression model
data['predictions'] = cross_val_predict(LogisticRegression(), X ,data['price_range_encoded'])

# Accuracy
print('Accuracy =', round(accuracy_score(data['price_range_encoded'], data['predictions']), 3))
# Precision
print('Precision =', round(precision_score(data['price_range_encoded'], data['predictions']), 3))
# Recall
print('Recall =', round(recall_score(data['price_range_encoded'], data['predictions']), 3))
# F1 score
print('F1 score =', round(f1_score(data['price_range_encoded'], data['predictions']), 3))